In [1]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import bloscpack as bp

import sys
import os
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from data_manager import *
from metadata import *

from annotation_utilities import *

from itertools import izip
import pandas as pd

import time

import matplotlib.pyplot as plt
%matplotlib inline

from vis3d_utilities import *

Setting environment for Local Macbook Pro


# Show annotated stacks

In [58]:
stack = 'MD589'

In [61]:
shell_polydata_rel2MD589uncropped = DataManager.load_shell_mesh(stack='MD589', downscale=32)
shell_actor_rel2MD589uncropped = actor_mesh(shell_polydata_rel2MD589uncropped, (1,1,1), opacity=.1, wireframe=False)

In [76]:
composite_atlasvol = np.zeros_like(atlasvol, np.float32)
composite_atlasvol_opacity = np.zeros_like(atlasvol, np.float32)

vol_all = None
vol_opacity_all = None

vol_bbox_rel2uncropped = DataManager.load_original_volume_bbox(stack=stack, volume_type='annotationAsScore', 
                                                               structure='7N_L', downscale=32, relative_to_uncropped=True)

# for name_s in all_known_structures_sided:
for name_s in ['7N_L', '7N_R']:
    vol = DataManager.load_original_volume(stack=stack, volume_type='annotationAsScore', structure=name_s, downscale=32)
    if vol_all is None:
        vol_all = vol
    else:
        vol_all = np.maximum(vol_all, vol)
        
    vol_opacity = np.zeros_like(vol, np.float32)
    vol_opacity[vol > 0.5] = 1.
    if vol_opacity_all is None:
        vol_opacity_all = vol_opacity
    else:
        vol_opacity_all = np.maximum(vol_opacity_all, vol_opacity)
    
launch_vtk([shell_actor_rel2MD589uncropped, 
        actor_volume(vol_all, what='score', 
                     auxdata=vol_opacity_all, 
                     origin=vol_bbox_rel2uncropped[[0,2,4]])],
      view_up=(.3,-1,0),
       position=(-4,-3,-6),
       focal=(0,0,0),
    axes=False)

# Show atlas as fuzzy score volumes

In [2]:
atlas_name = 'atlasV5'

canonical_centroid_rel2MD589uncropped = np.loadtxt(DataManager.get_atlas_canonical_centroid_filepath(atlas_name))
atlasvol_bbox_rel2MD589uncropped = DataManager.load_original_volume_bbox(stack=atlas_name, volume_type='score', 
                                                                         structure='7N_L', downscale=32, relative_to_uncropped=False)
atlasvol_bbox_rel2canon = atlasvol_bbox_rel2MD589uncropped - canonical_centroid_rel2MD589uncropped[[0,0,1,1,2,2]]

In [3]:
shell_polydata_rel2MD589uncropped = DataManager.load_shell_mesh(stack='MD589', downscale=32)
shell_actor_rel2canon = actor_mesh(shell_polydata_rel2MD589uncropped, (1,1,1), opacity=.1, wireframe=False, 
                                   origin=-canonical_centroid_rel2MD589uncropped)

In [4]:
# name_sided_to_value = {name_s: i % len(all_known_structures_sided) + 1.5
#          for i, name_s in enumerate(all_known_structures_sided)}
# color_transfer_func = {i % len(all_known_structures_sided) + 1.5: np.array(name_sided_to_color[name_s])/255.
#          for i, name_s in enumerate(all_known_structures_sided)}

name_unsided_to_value = {name_u: i % len(all_known_structures) + 1.5
         for i, name_u in enumerate(all_known_structures)}
color_transfer_func = {i % len(all_known_structures) + 1.5: np.array(name_unsided_to_color[name_u])/255.
         for i, name_u in enumerate(all_known_structures)}

In [5]:
atlasvol = DataManager.load_original_volume(stack=atlas_name, structure='7N_L', downscale=32)

In [6]:
composite_atlasvol = np.zeros_like(atlasvol, np.float32)
composite_atlasvol_opacity = np.zeros_like(atlasvol, np.float32)

for name_s in all_known_structures_sided:
# for name_s in ['7N_L', '7N_R']:
    
    atlasvol = DataManager.load_original_volume(stack=atlas_name, structure=name_s, downscale=32)
    composite_atlasvol[atlasvol > 0.01] = name_unsided_to_value[convert_to_original_name(name_s)]
    composite_atlasvol_opacity[atlasvol > 0.01] = atlasvol[atlasvol > 0.01]*0.1

In [7]:
launch_vtk([shell_actor_rel2canon, 
            actor_volume(composite_atlasvol, what='score', 
                         auxdata=composite_atlasvol_opacity, 
                         origin=atlasvol_bbox_rel2canon[[0,2,4]],
                         tb_colors=color_transfer_func)],
          view_up=(.3,-1,0),
           position=(-4,-3,-6),
           focal=(0,0,0),
        axes=False)

# Show subject

In [11]:
stack = 'MD589'
detector_id = 19

In [12]:
shell_polydata = DataManager.load_mesh_v2(brain_spec={'name':stack, 'resolution':'10.0um', 'vol_type':'annotationAsScore'}, structure='shell', return_polydata_only=True)
shell_actor = actor_mesh(shell_polydata, (1,1,1), opacity=.3, wireframe=False)

In [22]:
score_show_limit = 0.85
outside_roi_opacity = 0.01
in_roi_opacity = 0.1
# ROI is the bounding box for the structure.

# for structure_u in all_known_structures:
# for structure_u in ['SNR']:
for structure_s in ['7N_L']:

    vol, origin = DataManager.load_original_volume_v2(stack_spec=dict(name=stack,
                                                              vol_type='score',
                                                             detector_id=detector_id,
                                                             resolution='10.0um'),
                                              structure=structure_s)

#     scvol_bbox_rel2uncropped = \
#     DataManager.load_original_(stack=stack, volume_type='score', 
#                                           structure=structure_u, 
#                                           prep_id=2, detector_id=detector_id, downscale=32,
#                                          relative_to_uncropped=True)

#     annvols, _, _ = DataManager.load_original_volume_all_known_structures(stack=stack, structures= structure_u if structure_u in singular_structures else [convert_to_left_name(structure_u), convert_to_right_name(structure_u)], 
#                                                                           downscale=32, 
#                                      volume_type='annotationAsScore')
#     annvol_bbox_rel2uncropped = DataManager.load_original_volume_bbox(stack=stack, structure='7n_R', downscale=32, 
#                                      volume_type='annotationAsScore')


    toshow_xyzs_rel2sc = np.array(np.nonzero(vol > score_show_limit))[[1,0,2]].T # these are shown with lower opacity

    vol_opacity = np.zeros(vol.shape, np.float32)
    for x_rel2sc, y_rel2sc, z_rel2sc in toshow_xyzs_rel2sc:
        vol_opacity[y_rel2sc, x_rel2sc, z_rel2sc] = in_roi_opacity

#         vol_opacity[y_rel2sc, x_rel2sc, z_rel2sc] = outside_roi_opacity

#         xyz_rel2ann = (x_rel2sc, y_rel2sc, z_rel2sc) + scvol_bbox_rel2uncropped[[0,2,4]] - annvol_bbox_rel2uncropped[[0,2,4]]
#         for annvol in annvols.itervalues():
#             if all(xyz_rel2ann > 0) and \
#             xyz_rel2ann[0] < annvol.shape[1] and \
#             xyz_rel2ann[1] < annvol.shape[0] and \
#             xyz_rel2ann[2] < annvol.shape[2] and \
#             annvol[xyz_rel2ann[1], xyz_rel2ann[0], xyz_rel2ann[2]] > 0:
#                 vol_opacity[y_rel2sc, x_rel2sc, z_rel2sc] = in_roi_opacity

    launch_vtk([shell_actor, 
#                 actor_sphere((-40,0,-60), radius=10, color=(0,1,0)),
#                 actor_sphere((0,0,0), radius=10, color=(1,0,0)),
                actor_volume(vol.astype(np.float32), 'score', 
                                          origin=origin,
                                         auxdata=vol_opacity,
                                          c=(0,0,0)
                                         )], 
               view_up=(.3,-1,0),
               position=(-4,-3,-6),
               focal=(0,0,0),
               background_color=(1,1,1),
               interactive=True, 
#                 interactive=False,
#                snapshot_fn='/home/yuncong/CSHL_scorevolume_viz/%s.png' % structure_u,
#                axes=True,
               axes=False
              )